In [3]:
import math
import numpy as np

In [5]:
def load_coffee_data():
    rng = np.random.default_rng(2)
    X = rng.random(400).reshape(-1,2)
    X[:,1] = X[:,1] * 4 + 11.5          # 12-15 min is best
    X[:,0] = X[:,0] * (285-150) + 150  # 350-500 F (175-260 C) is best
    Y = np.zeros(len(X))
    
    i=0
    for t,d in X:
        y = -3/(260-175)*t + 21
        if (t > 175 and t < 260 and d > 12 and d < 15 and d<=y ):
            Y[i] = 1
        else:
            Y[i] = 0
        i += 1

    return (X, Y.reshape(-1,1))

In [7]:
def ReLU(z):
    return np.maximum(0, z)

In [9]:
def sigmoid(z):
    g = 1 / (1 + np.exp(-z))
    return g

In [11]:
def compute_cost(Y, model):
    epsilon = 1e-8  # to prevent log(0)
    loss = Y * np.log(model + epsilon) + (1 - Y) * np.log(1 - model + epsilon)
    return -np.mean(loss)

In [13]:
def forward(X, W, B, layers):
    l = len(layers)
    Z = [0.] * l
    A = [0.] * (l+1) #[X, A1, A2]
    A[0] = X
    for i in range(l):
        Z[i] = A[i] @ W[i] + B[i]
        if layers[i] == "ReLU":
            A[i+1] = ReLU(Z[i])
        elif layers[i] == "Sigmoid":
            A[i+1] = sigmoid(Z[i])
        elif layers[i] == "Linear":
            A[i+1] = Z[i]
    return A, Z

In [15]:
def backprop(layers, X, Y, A, B, W, Z):
    #layers = ["ReLU", "Sigmoid"]
    #W = [W_1, W_2]
    #B = [B_1, B_2]
    #C = [C_1, C_2]
    #A = [A_0, A_1, A_2]
    #Z = [Z_1, Z_2]
    m = X.shape[0]
    loss = compute_cost(Y, A[-1])
    dJ_dW = [0.]*len(layers)
    dJ_dB = [0.]*len(layers)
    dJ_dA = - (Y/A[-1] - (1-Y)/(1-A[-1]))
    for l in reversed(range(len(layers))):
        if layers[l] == "ReLU":
            dA_dZ = (Z[l] > 0).astype(float) 
        elif layers[l] == "Sigmoid":
            dA_dZ = sigmoid(Z[l]) * (1-sigmoid(Z[l]))
        elif layers[l] == "Linear":
            dA_dZ = np.ones_like(Z[l])
        dJ_dZ = dJ_dA * dA_dZ
        dJ_dC = dJ_dZ
        dJ_dB[l] = np.sum(dJ_dZ, axis=0, keepdims=True) / m
        dC_dA = W[l]; dC_dW = A[l]
        dJ_dA = dJ_dC @ W[l].T
        dJ_dW[l] = (dC_dW.T @ dJ_dC) / m 
    return dJ_dW, dJ_dB

In [17]:
def gradient_descent(X, Y, W_in, B_in, num_liters, alpha, layers):
    m, n = X.shape
    W = W_in #np.array() 1-D
    B = B_in #np.array() 1-D
    l = len(layers)
    A, Z = forward(X, W, B, layers)
    for i in range(num_liters):
        dJ_dW, dJ_dB = backprop(layers, X, Y, A, B, W, Z)
        for j in range(l):
            W[j] = W[j] - alpha * dJ_dW[j]
            B[j] = B[j] - alpha * dJ_dB[j]
        A, Z = forward(X, W, B, layers)
        J = compute_cost(Y, A[-1])
        if i % math.ceil(num_liters/10) == 0:
            print(f'At {i}, cost function(J) = {J}')
    return W, B

In [19]:
def normalized(X):
    mu = np.mean(X, axis = 0)
    sigma = np.std(X, axis = 0)

    norm_X = (X-mu)/sigma
    return norm_X, mu, sigma

In [21]:
def prediction(y_hat):
    return (y_hat > 0.5).astype(int)

In [1]:
X, Y = load_coffee_data()

NameError: name 'load_coffee_data' is not defined

In [25]:
X.shape, Y.shape

((200, 2), (200, 1))

In [27]:
#plan
#input: X (200, 2)
#Layer1 : 3 neurons, W_1 (2, 3) B_1 (3,) => output: A_1 (200, 3) ; activation = ReLU
#Layer2 : 1 neurons, W_2 (3, 1) B_2 (1,) => output: A_2 (200, 1) ; activation = Sigmoid

In [29]:
X_train = X[:160]; Y_train = Y[:160]
X_test = X[160:]; Y_test = Y[160:] 
W_1_in = np.random.randn(2, 3) * 0.01
B_1_in = np.zeros((3,))
W_2_in = np.random.randn(3, 1) * 0.01
B_2_in = np.zeros((1,))
W = [W_1_in, W_2_in]
B = [B_1_in, B_2_in]

In [31]:
norm_X_train, mu, sigma = normalized(X_train)

In [33]:
norm_X_train[:2]

array([[-0.81483957, -0.6340911 ],
       [ 1.06004726, -1.364514  ]])

In [51]:
num = 10000
alpha = 1.0e-1
layers = ["ReLU", "Sigmoid"]

W_last, B_last = gradient_descent(norm_X_train, Y_train, W, B, num, alpha, layers)

At 0, cost function(J) = 0.0030606047027810657
At 1000, cost function(J) = 0.0030292306339489796
At 2000, cost function(J) = 0.0029984629252984263
At 3000, cost function(J) = 0.0029682844350143603
At 4000, cost function(J) = 0.002938678661322723
At 5000, cost function(J) = 0.002909629711680424
At 6000, cost function(J) = 0.0028811222744228445
At 7000, cost function(J) = 0.0028531415921830014
At 8000, cost function(J) = 0.00282567343683372
At 9000, cost function(J) = 0.002798704085818311


In [53]:
norm_X_test = (X_test - mu) / sigma

In [61]:
W_last, B_last

([array([[-7.68619626,  8.52389134,  0.97309671],
         [-0.43056969,  7.48970029, -5.54866459]]),
  array([[-10.48745964],
         [-11.46239182],
         [ -7.74102841]])],
 [array([[-7.12213836,  1.62403004, -5.30967404]]), array([[13.44611012]])])

In [63]:
A, _ = forward(norm_X_test, W_last, B_last, layers)
Y_hat = A[-1]

In [65]:
accuracy = (prediction(Y_hat) == Y_test).mean()
accuracy*100 

92.5

In [67]:
prediction(Y_hat[:5])

array([[0],
       [1],
       [0],
       [0],
       [0]])

In [69]:
Y_test[:5]

array([[0.],
       [1.],
       [0.],
       [0.],
       [0.]])